In [1]:
!pip install fastf1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.0/165.0 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.17.0 requires websockets<16.0.0,>=15.0.1, but you have websockets 13.1 which is incompatible.
dataproc-spark-connect 0.8.3 requires websockets>=14.0, but you have websockets 13.1 which is incompatible.


In [2]:
import fastf1
import pandas as pd
import matplotlib.pyplot as plt
import os

# Enable Cache
if not os.path.exists('cache'):
  os.makedirs('cache')
fastf1.Cache.enable_cache('cache')
session = fastf1.get_session(2025, 'United States Grand Prix', 'R')
session.load()

laps = session.laps

# Filter only the laps 47–51 for both drivers
lando_laps = laps[(laps['Driver'] == 'NOR') & (laps['LapNumber'] >= 47) & (laps['LapNumber'] <= 51) & laps['LapTime'].notna()].copy()
charles_laps = laps[(laps['Driver'] == 'LEC') & (laps['LapNumber'] >= 47) & (laps['LapNumber'] <= 51) & laps['LapTime'].notna()].copy()

# Drop pit laps safely
for name, df in [('Lando', lando_laps), ('Charles', charles_laps)]:
    if 'PitOutLap' in df.columns:
        df = df[~df['PitOutLap']]
    if 'PitInLap' in df.columns:
        df = df[~df['PitInLap']]
    if name == 'Lando':
        lando_laps = df
    else:
        charles_laps = df

# Convert LapTime to seconds
lando_laps.loc[:, 'LapTime_s'] = lando_laps['LapTime'].dt.total_seconds()
charles_laps.loc[:, 'LapTime_s'] = charles_laps['LapTime'].dt.total_seconds()

# Merge both drivers’ lap times on LapNumber
comparison = pd.merge(
    lando_laps[['LapNumber', 'LapTime_s']],
    charles_laps[['LapNumber', 'LapTime_s']],
    on='LapNumber',
    suffixes=('_Norris', '_Leclerc')
)

# Calculate per-lap delta and cumulative gain
comparison['Delta_per_lap'] = comparison['LapTime_s_Leclerc'] - comparison['LapTime_s_Norris']
comparison['Cumulative_gain'] = comparison['Delta_per_lap'].cumsum()

# Round for readability
comparison = comparison.round(3)

print(comparison[['LapNumber', 'LapTime_s_Norris', 'LapTime_s_Leclerc', 'Delta_per_lap', 'Cumulative_gain']])


events      WARNING 	Correcting user input 'United States Grand Prix' to 'United States Grand Prix'
core           INFO 	Loading data for United States Grand Prix - Race [v3.6.1]
INFO:fastf1.fastf1.core:Loading data for United States Grand Prix - Race [v3.6.1]
req            INFO 	No cached data found for session_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
INFO:fastf1.api:Fetching session info data...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
INFO:fastf1.api:Fetching driver list...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data

   LapNumber  LapTime_s_Norris  LapTime_s_Leclerc  Delta_per_lap  \
0       47.0            97.735             98.231          0.496   
1       48.0            97.869             98.531          0.662   
2       49.0            98.379             98.684          0.305   
3       50.0            98.495             98.820          0.325   
4       51.0            99.118            100.437          1.319   

   Cumulative_gain  
0            0.496  
1            1.158  
2            1.463  
3            1.788  
4            3.107  
